High Value Customer Identification (Insiders)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [2]:
data = pd.read_csv("df_products.csv")
data.head(2)

,product_id,product_category,product_name,product_price,scrapy_datetime,additional material information,art. no.,care instructions,collection,composition,concept,description,fit,imported,material,size,style_id,color_id
0,690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-29 16:18:45,NaN,690449043,Only non-chlorine bleach when needed,NaN,"Cotton 98%, Spandex 2%",DENIM,Light denim blue,Skinny fit,Yes,keyFibreType,NaN,690449,43
1,690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-11-29 16:18:45,NaN,690449043,Medium iron,NaN,"Pocket lining: Polyester 65%, Cotton 35%",DENIM,Light denim blue,Skinny fit,Yes,Denim,NaN,690449,43


In [7]:
data2 = data.copy()

for i in ['style_id', 'color_id', 'product_id']:
    data2[i] = data2[i].astype(int)

for i in ['product_name', 'fit', 'description', 'material']:
    data2[i] = data2[i].apply( lambda x: x.replace(" ", "_").lower())
    data2[i] = data2[i].apply( lambda x: x.replace("/", "_").lower())

data2['composition'] = data2['composition'].str.lower()

data2['product_price'] = data2['product_price'].apply( lambda x: x.replace("$", "")).astype(float)

data2['scrapy_datetime'] = pd.to_datetime(data2['scrapy_datetime'], format = "%Y-%m-%d %H:%M:%S")

data2['size_number'] = data2['size'].apply( lambda x: re.search("\d+[^cm]*", x).group(0) if pd.notnull( x ) else x )

data2['size_model'] = data2['size'].str.extract( '(\d+\/\d+)' )

data2 = data2.drop(['size'], axis=1)


print(data2.shape)

for i in ['lining', 'shell', 'pocket']:
    data2 = data2[~data2['composition'].str.contains( i, na=False) ]

print(data2.shape)


data2.head(2)

(67, 19)
(8, 19)


,product_id,product_category,product_name,product_price,scrapy_datetime,additional material information,art. no.,care instructions,collection,composition,concept,description,fit,imported,material,style_id,color_id,size_number,size_model
0,690449043,men_jeans_ripped,skinny_jeans,39.99,2022-11-29 16:18:45,NaN,690449043,Only non-chlorine bleach when needed,NaN,"cotton 98%, spandex 2%",DENIM,light_denim_blue,skinny_fit,Yes,keyfibretype,690449,43,NaN,NaN
16,690449022,men_jeans_ripped,skinny_jeans,39.99,2022-11-29 16:18:45,NaN,690449022,Only non-chlorine bleach when needed,NaN,"cotton 98%, spandex 2%",DENIM,black,skinny_fit,Yes,keyfibretype,690449,22,NaN,NaN


In [10]:
df_ref = pd.DataFrame(index=np.arange(len(data2)) , columns=['cotton', 'polyester', 'elastane', 'elasterell'])
df_ref.head(2)

,cotton,polyester,elastane,elasterell
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN


In [11]:
df1 = data2['composition'].str.split( ',', expand=True )

In [12]:
# cotton | polyester | elastano | elasterell
df_ref = pd.DataFrame( index=np.arange( len( data ) ), columns=['cotton','polyester', 'elastane', 'elasterell'] )
# cotton
df_cotton = df1[0]
df_cotton.name = 'cotton'
df_ref = pd.concat( [df_ref, df_cotton ], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]
# polyester
df_polyester = df1.loc[df1[1].str.contains( 'Polyester', na=True ), 1]
df_polyester.name = 'polyester'
df_ref = pd.concat( [df_ref, df_polyester], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]
# elastano
df_elastane = df1.loc[df1[1].str.contains( 'Elastane', na=True ), 1]

In [13]:
df_elastane.name = 'elastane'
df_ref = pd.concat( [df_ref, df_elastane], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]
# elasterell
df_elasterell = df1.loc[df1[1].str.contains( 'Elasterell', na=True ), 1]
df_elasterell.name = 'elasterell'
df_ref = pd.concat( [df_ref, df_elasterell], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]
# final join
df1 = pd.concat( [df1, df_ref], axis=1 )
# format composition df1
df1['cotton'] = df1['cotton'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df1['polyester'] = df1['polyester'].apply( lambda x: int( re.search( '\d+', x).group(0) ) / 100 if pd.notnull( x ) else x )
df1['elastane'] = df1['elastane'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df1['elasterell'] = df1['elasterell'].apply( lambda x: int( re.search( '\d+',x ).group(0) ) / 100 if pd.notnull( x ) else x )
df1.head()


,0,1,2,cotton,polyester,elastane,elasterell
0,cotton 98%,spandex 2%,None,0.98,NaN,NaN,NaN
16,cotton 98%,spandex 2%,None,0.98,NaN,NaN,NaN
24,cotton 98%,spandex 2%,None,0.98,NaN,NaN,NaN
35,cotton 98%,spandex 2%,None,0.98,NaN,NaN,NaN
38,cotton 98%,spandex 2%,None,0.98,NaN,NaN,NaN


In [14]:
df1.head(20)

,0,1,2,cotton,polyester,elastane,elasterell
0,cotton 98%,spandex 2%,None,0.98,NaN,NaN,NaN
16,cotton 98%,spandex 2%,None,0.98,NaN,NaN,NaN
24,cotton 98%,spandex 2%,None,0.98,NaN,NaN,NaN
35,cotton 98%,spandex 2%,None,0.98,NaN,NaN,NaN
38,cotton 98%,spandex 2%,None,0.98,NaN,NaN,NaN
40,cotton 79%,polyester 20%,spandex 1%,0.79,NaN,NaN,NaN
59,cotton 78%,polyester 21%,spandex 1%,0.78,NaN,NaN,NaN
60,cotton 99%,spandex 1%,None,0.99,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in ['cotton', 'polyester', 'elastano', 'elasterell']:
    mask = data2.apply(lambda x: True if i in x else False, axis=1)
mask

0     False
16    False
24    False
35    False
38    False
40    False
59    False
60    False
dtype: bool

In [ ]:
df1.head(10)

,0,1,2
0,cotton 98%,spandex 2%,None
16,cotton 98%,spandex 2%,None
24,cotton 98%,spandex 2%,None
35,cotton 98%,spandex 2%,None
38,cotton 98%,spandex 2%,None
40,cotton 79%,polyester 20%,spandex 1%
59,cotton 78%,polyester 21%,spandex 1%
60,cotton 99%,spandex 1%,None
